# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [4]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [5]:
learning_rate=0.02
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SimpleNet(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_layers=2, num_classes=10):
        super(SimpleNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


# 定义模型
model = SimpleNet().to(device)

# 损失函数和优化算法
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Next, we can start to train and evaluate!

In [6]:
# 训练模型
total_step = len(train_loader)
minloss=1
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28, 28).to(device)
        labels = labels.to(device)

        # 前向传播+计算loss
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 后向传播+调整参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 每100个batch打印一次数据
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, NUM_EPOCHS, i + 1, total_step, loss.item()))
        if loss.item()<minloss:
            minloss=loss.item()
print("The training accuracy is: ",(1-minloss)*100,"%")

# 模型测试部分
# 测试阶段不需要计算梯度，注意
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28, 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('The testing accuracy is: {} %'.format(100 * correct / total))

Epoch [1/10], Step [100/468], Loss: 0.3751
Epoch [1/10], Step [200/468], Loss: 0.2237
Epoch [1/10], Step [300/468], Loss: 0.2084
Epoch [1/10], Step [400/468], Loss: 0.1630
Epoch [2/10], Step [100/468], Loss: 0.2837
Epoch [2/10], Step [200/468], Loss: 0.4219
Epoch [2/10], Step [300/468], Loss: 0.5196
Epoch [2/10], Step [400/468], Loss: 0.4486
Epoch [3/10], Step [100/468], Loss: 0.3339
Epoch [3/10], Step [200/468], Loss: 0.3295
Epoch [3/10], Step [300/468], Loss: 0.5432
Epoch [3/10], Step [400/468], Loss: 0.2957
Epoch [4/10], Step [100/468], Loss: 0.4274
Epoch [4/10], Step [200/468], Loss: 0.4537
Epoch [4/10], Step [300/468], Loss: 0.2409
Epoch [4/10], Step [400/468], Loss: 0.2177
Epoch [5/10], Step [100/468], Loss: 0.3817
Epoch [5/10], Step [200/468], Loss: 0.1774
Epoch [5/10], Step [300/468], Loss: 0.6464
Epoch [5/10], Step [400/468], Loss: 0.2417
Epoch [6/10], Step [100/468], Loss: 0.4406
Epoch [6/10], Step [200/468], Loss: 0.2620
Epoch [6/10], Step [300/468], Loss: 0.4457
Epoch [6/10

#### Q5:
Please print the training and testing accuracy.